In [1]:
# Importación de librerías necesarias
import pandas as pd
from neuralprophet import NeuralProphet
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
#
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')
productos_descripcion = pd.read_csv('data/tb_productos_descripcion.txt', delimiter='\t')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1600)

# Muestra las primeras filas del DataFrame
print(data_full.head())
data_full.info()
print(df_pid_validos.head())
print(productos_descripcion.head())
productos_descripcion.info()

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_

In [3]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [4]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
filtered_data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(filtered_data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = filtered_data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = filtered_data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

# Ver como esta el data frame
filtered_data.info()
filtered_data

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452
...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230
2945814,201912,10092,20853,0,1,0.00669,0.00669
2945815,201912,10006,20853,0,7,0.02898,0.02898
2945816,201912,10018,20853,0,4,0.01561,0.01561


In [5]:
# Función para filtrar datos por rango de product_id
def filtrar_por_producto(df, producto_inicio, producto_fin):
    return df[(df['product_id'] >= producto_inicio) & (df['product_id'] <= producto_fin)]

# Filtrado de datos para el rango de productos deseado
producto_inicio = 20001
#producto_fin = 21276
producto_fin = 20012
datos_filtrados = filtrar_por_producto(filtered_data, producto_inicio, producto_fin)

# Preparación de datos para NeuralProphet
def preparar_datos(df):
    df = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
    df['ds'] = pd.to_datetime(df['periodo'], format='%Y%m')
    df.rename(columns={'tn': 'y'}, inplace=True)
    return df[['ds', 'y', 'product_id']]

datos_preparados = preparar_datos(datos_filtrados)

# Normalización de los datos
scaler = StandardScaler()
datos_preparados['y'] = scaler.fit_transform(datos_preparados[['y']])

# División de los datos en train, validation y test
def dividir_datos(df, train_frac=0.7, val_frac=0.2):
    train_size = int(len(df) * train_frac)
    val_size = int(len(df) * val_frac)
    train_df = df[:train_size]
    val_df = df[train_size:train_size+val_size]
    test_df = df[train_size+val_size:]
    return train_df, val_df, test_df

train_df, val_df, test_df = dividir_datos(datos_preparados)

# Entrenamiento y predicción con NeuralProphet
def entrenar_y_predecir(train_df, val_df, test_df, periodo_prediccion):
    modelos = {}
    predicciones = pd.DataFrame()
    
    for producto in train_df['product_id'].unique():
        m.fit(df_producto_train, validation_df=df_producto_val, freq='M')
        
        future = m.make_future_dataframe(df_producto_test, periods=periodo_prediccion)
        forecast = m.predict(future)
        
        forecast['yhat1'] = scaler.inverse_transform(forecast[['yhat1']])
        forecast['product_id'] = producto
        predicciones = pd.concat([predicciones, forecast[['ds', 'yhat1', 'product_id']]])
        modelos[producto] = m
    
    return predicciones, modelos    df_producto_train = train_df[train_df['product_id'] == producto][['ds', 'y']]
        df_producto_val = val_df[val_df['product_id'] == producto][['ds', 'y']]
        df_producto_test = test_df[test_df['product_id'] == producto][['ds', 'y']]
        
        m = NeuralProphet(loss_func='Huber')
      

# Configuración de la predicción
periodo_prediccion = 2 # Predecir 2 meses para llegar a febrero 2020

# Entrenar el modelo y obtener predicciones
predicciones, modelos = entrenar_y_predecir(train_df, val_df, test_df, periodo_prediccion)

# Filtrar las predicciones para el mes de febrero 2020
predicciones['periodo'] = predicciones['ds'].dt.strftime('%Y%m').astype(int)
predicciones_febrero_2020 = predicciones[predicciones['periodo'] == 202002][['product_id', 'yhat1']]

# Renombrar las columnas para el archivo de salida
predicciones_febrero_2020.rename(columns={'yhat1': 'tn'}, inplace=True)

# Guardar el resultado en un archivo .csv
predicciones_febrero_2020.to_csv('NeuralProphet.20013al20025.v7.0.2.csv', index=False)

# Mostrar el resultado
print(predicciones_febrero_2020)

IndentationError: unexpected indent (3143834063.py, line 43)